In [5]:
import os
import json
import uuid
import requests
import gradio as gr
import pandas as pd

from datetime import datetime

In [6]:
ingredients = []
table_columns = ['ingredient_id', 'name', 'quantity', 'purchase_date', 'expiration_date']

def add_ingredients(**kwargs):
    ingredient_id = str(uuid.uuid4())
    
    ingredients.append({
        'ingredient_id': ingredient_id,
        **kwargs
    })
    

def update_ingredient(ingredient_id, **kwargs):
    for i in range(len(ingredients)):
        if ingredients[i]['ingredient_id'] == ingredient_id:
            ingredients[i].update(kwargs)


def delete_ingredient(ingredient_id):
    for i in range(len(ingredients)):
        if ingredients[i]['ingredient_id'] == ingredient_id:
            return ingredients.pop(i)
    return None


def get_ingredients_df():
    return pd.DataFrame(ingredients, columns=table_columns)

In [ ]:
SPEECH_API_KEY = '***'
GPT_API_KEY = '***'


def request_stt(file_path):
    if not file_path:
        return

    ENDPOINT = 'https://eastus.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?language=ko-KR&format=detailed'
    headers = {
        'Ocp-Apim-Subscription-Key': SPEECH_API_KEY,
        'Content-Type': 'audio/wav',
        'language': 'ko-KR',
        'format': 'detailed'
    }
    with open(file_path, 'rb') as f:
        data = f.read()

    response = requests.post(ENDPOINT, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['DisplayText']


def request_tts(text, speed, voice):
    if not text or text.startswith('⚠️'):
        return

    ENDPOINT = 'https://eastus.tts.speech.microsoft.com/cognitiveservices/v1'
    headers = {
        'Ocp-Apim-Subscription-Key': SPEECH_API_KEY,
        'Content-Type': 'application/ssml+xml',
        'X-Microsoft-OutputFormat': 'audio-16khz-128kbitrate-mono-mp3',
    }
    data = f"""
    <speak version='1.0' xml:lang='ko-KR'>
        <voice xml:lang='ko-KR' xml:gender='Female' name='{voice}'>
            <prosody rate='{speed}'>
                {text}
            </prosody>
        </voice>
    </speak>
    """

    response = requests.post(ENDPOINT, headers=headers, data=data)
    if response.status_code == 200:
        os.makedirs('./resources', exist_ok=True)
        with open('./resources/tmp.mp3', 'wb') as f:
            f.write(response.content)
        return './resources/tmp.mp3'


def request_gpt(history):
    if not history:
        return history
    if not history[-1]['content']:
        history.pop()
        return history

    ENDPOINT = 'https://fimtrus-openai3.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21'
    SYSTEM_INSTRUCTION = {
        'role': 'system',
        'content': '당신은 사용자의 냉장고의 식자재 관리를 도와주는 AI 음성 도우미입니다. '
                   '이모지나 마크다운 등을 사용하지 말고, 소리내서 발음할 수 있도록 답하세요. '
                   '사용자가 어떤 식자재를 냉장고에 넣었고, 어떤 것을 소비했는지, '
                   '각각의 구매일자와 소비 기한이 어떻게 되는지를 파악하는 것이 당신의 역할입니다. '
                   '만약 사용자가 구매 일자를 명시적으로 언급하지 않는다면 메시지 수신 시각을 구매 일자로 저장하세요. '
                   '만약 사용자가 소비 기한을 명시적으로 언급하지 않는다면 해당 식자재의 보편적인 유통기한을 토대로 소비 일자를 계산해 저장하세요. '
                   '저장하는 내용은 반드시 사용자에게 설명하세요. '
                   '만약 식자재를 수정하거나 삭제해야 한다면, 메시지의 뒤에 첨부된 전체 식자재 현황 데이터에 실제로 존재하는 `ingredient_id`를 사용하여 '
                   '어떤 식자재에 대한 수정이나 삭제인지 명시하세요.\n\n'
                   '!경고: 식자재가 아닌 물건은 등록하지 마세요.'
    }
    headers = {
        'Content-Type': 'application/json',
        'API-Key': GPT_API_KEY
    }

    original_messsage = history[-1]['content']

    history[-1]['content'] += f'\n\n메시지 수신 시각: {datetime.now()}\n\n현재 식자재 현황: {ingredients}'

    print('@hidden...', history[-1])

    json_data = {
        'messages': [SYSTEM_INSTRUCTION] + history,
        'temperature': 0.7,
        'top_p': 0.9,
        'max_tokens': 800,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "IngredientManagementResponse",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "items": {
                            "type": "array",
                            "description": "A list of ingredient management actions to be performed.",
                            "items": {
                                "anyOf": [
                                    {
                                        "type": "object",
                                        "description": "Add a new ingredient to the database",
                                        "properties": {
                                            "action": {
                                                "type": "string",
                                                "enum": ["add"],
                                                "description": "Defines that this action adds a new ingredient."
                                            },
                                            "details": {
                                                "type": "object",
                                                "description": "Details of the ingredient to be added.",
                                                "properties": {
                                                    "name": {
                                                        "type": "string",
                                                        "description": "The name of the ingredient."
                                                    },
                                                    "quantity": {
                                                        "type": "string",
                                                        "description": "The quantity of the ingredient."
                                                    },
                                                    "purchase_date": {
                                                        "type": "string",
                                                        "description": "The purchase date of the ingredient."
                                                    },
                                                    "expiration_date": {
                                                        "type": "string",
                                                        "description": "The expiration date of the ingredient."
                                                    }
                                                },
                                                "required": ["name", "quantity", "purchase_date", "expiration_date"],
                                                "additionalProperties": False
                                            }
                                        },
                                        "required": ["action", "details"],
                                        "additionalProperties": False
                                    },
                                    {
                                        "type": "object",
                                        "description": "Update an existing ingredient in the database",
                                        "properties": {
                                            "action": {
                                                "type": "string",
                                                "enum": ["update"],
                                                "description": "Defines that this action updates an existing ingredient."
                                            },
                                            "ingredient_id": {
                                                "type": "string",
                                                "description": "The unique ID of the ingredient to update."
                                            },
                                            "details": {
                                                "type": ["object", "null"],
                                                "description": "Details of the ingredient to update.",
                                                "properties": {
                                                    "name": {
                                                        "type": "string",
                                                        "description": "The updated name of the ingredient (optional)."
                                                    },
                                                    "quantity": {
                                                        "type": "string",
                                                        "description": "The updated quantity of the ingredient (optional)."
                                                    },
                                                    "purchase_date": {
                                                        "type": "string",
                                                        "description": "The updated purchase date of the ingredient (optional)."
                                                    },
                                                    "expiration_date": {
                                                        "type": "string",
                                                        "description": "The updated expiration date of the ingredient (optional)."
                                                    }
                                                },
                                                "required": ["name", "quantity", "purchase_date", "expiration_date"],
                                                "additionalProperties": False
                                            }
                                        },
                                        "required": ["action", "ingredient_id", "details"],
                                        "additionalProperties": False
                                    },
                                    {
                                        "type": "object",
                                        "description": "Delete an existing ingredient from the database",
                                        "properties": {
                                            "action": {
                                                "type": "string",
                                                "enum": ["delete"],
                                                "description": "Defines that this action deletes an existing ingredient."
                                            },
                                            "ingredient_id": {
                                                "type": "string",
                                                "description": "The unique ID of the ingredient to delete."
                                            }
                                        },
                                        "required": ["action", "ingredient_id"],
                                        "additionalProperties": False
                                    },
                                    {
                                        "type": "object",
                                        "description": "Take no action",
                                        "properties": {
                                            "action": {
                                                "type": "string",
                                                "enum": ["no_action"],
                                                "description": "Defines that no action should be performed."
                                            }
                                        },
                                        "required": ["action"],
                                        "additionalProperties": False
                                    }
                                ]
                            }
                        },
                        "response_message": {
                            "type": "string",
                            "description": "Message to communicate with the user about the actions taken or any additional information."
                        }
                    },
                    "required": ["items", "response_message"],
                    "additionalProperties": False
                }
            }
        }
    }

    history[-1]['content'] = original_messsage

    response = requests.post(ENDPOINT, headers=headers, json=json_data)
    if response.status_code == 200:
        raw_message = json.loads(
            response.json()['choices'][0]['message']['content'])
        print(raw_message)

        response_message = {
            'role': 'assistant',
            'content': raw_message['response_message']
        }

        for data in raw_message['items']:
            action = data['action']

            if action == 'add':
                add_ingredients(**data['details'])
            elif action == 'delete':
                delete_ingredient(data['ingredient_id'])
            elif action == 'update':
                update_ingredient(data['details'], **data['details'])
    else:
        response_message = {
            'role': 'assistant',
            'content': f'⚠️ 문제가 발생했습니다: [{response.status_code}] {response.text}'
        }
    history.append(response_message)
    return history

In [ ]:
history = []
ingredients = []

def create_user_message(message, history):
    history.append({
        'role': 'user',
        'content': message
    })
    return history


def get_last_message_content(history, *args):
    return history[-1]['content'] if history else '테스트 문장입니다.', *args


with gr.Blocks() as demo:
    gr.Markdown('# 🧊 식자재 관리 도우미')
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(history, type='messages')
        
        with gr.Column():
            voice_dropdown = gr.Dropdown(
                label='🗣️ 음성 선택', 
                choices=[
                    'ko-KR-SunHiNeural', 'ko-KR-InJoonNeural', 'ko-KR-HyunsuMultilingualNeural',
                    'ko-KR-BongJinNeural', 'ko-KR-GookMinNeural', 'ko-KR-HyunsuNeural',
                    'ko-KR-JiMinNeural', 'ko-KR-SeoHyeonNeural', 'ko-KR-SoonBokNeural',
                    'ko-KR-YuJinNeural'
                ], 
                interactive=True
            )
            speed_slider = gr.Slider(label='⏩ 발화 속도', minimum=0.1, maximum=2.0, value=1.0, interactive=True)
            tts_output = gr.Audio(label='합성 음성', type='filepath', interactive=False, autoplay=True)

    text_input = gr.Textbox(
        placeholder='메시지를 입력하거나 음성을 녹음해보세요.',
        show_label=False,
        submit_btn=True,
    )
    mic_input = gr.Microphone(show_label=False, type='filepath')

    gr.Markdown('<br>')
    gr.Markdown('## 📋 식자재 관리 현황')
    ingredient_df = gr.Dataframe(headers=table_columns)

    text_input.submit(
        create_user_message, inputs=[text_input, chatbot], outputs=[chatbot]
    ).then(
        request_gpt, inputs=[chatbot], outputs=[chatbot]
    ).then(
        lambda: (None, None), outputs=[text_input, mic_input]
    ).then(
        lambda *args: request_tts(*get_last_message_content(*args)), 
        inputs=[chatbot, speed_slider, voice_dropdown], outputs=[tts_output]
    ).then(
        get_ingredients_df,
        inputs=[], outputs=[ingredient_df]
    )

    mic_input.input(request_stt, inputs=[mic_input], outputs=[text_input])
    
    speed_slider.change(
        lambda *args: request_tts(*get_last_message_content(*args)), 
        inputs=[chatbot, speed_slider, voice_dropdown], outputs=[tts_output]
    )
    voice_dropdown.change(
        lambda *args: request_tts(*get_last_message_content(*args)), 
        inputs=[chatbot, speed_slider, voice_dropdown], outputs=[tts_output]
    )
    

demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


@hidden... {'role': 'user', 'metadata': None, 'content': '우유 한 병과 계란 한 판, 삼겹살 한 줄, 물티슈, 휴지, 아보카도 네 알, 방향제, 종량제 봉투 10장, 밀키트 두 팩 구매\n\n메시지 수신 시각: 2025-03-24 09:27:20.895673\n\n현재 식자재 현황: []', 'options': None}
{'items': [{'action': 'add', 'details': {'name': '우유', 'quantity': '1병', 'purchase_date': '2023-11-02', 'expiration_date': '2023-11-09'}}, {'action': 'add', 'details': {'name': '계란', 'quantity': '1판', 'purchase_date': '2023-11-02', 'expiration_date': '2023-11-16'}}, {'action': 'add', 'details': {'name': '삼겹살', 'quantity': '1줄', 'purchase_date': '2023-11-02', 'expiration_date': '2023-11-04'}}, {'action': 'add', 'details': {'name': '아보카도', 'quantity': '4알', 'purchase_date': '2023-11-02', 'expiration_date': '2023-11-06'}}, {'action': 'add', 'details': {'name': '밀키트', 'quantity': '2팩', 'purchase_date': '2023-11-02', 'expiration_date': '2023-11-09'}}], 'response_message': '우유, 계란, 삼겹살, 아보카도, 밀키트가 등록되었습니다. 물티슈, 휴지, 방향제, 종량제 봉투는 식자재가 아니므로 등록하지 않았습니다.'}
